In [2]:
from __future__ import division

import numpy as np
import os, glob, json
import matplotlib.pyplot as plt
import pickle
import scipy.linalg as sl
import healpy as hp
import multiprocessing as mp

from la_forge.core import Core
from la_forge.diagnostics import plot_chains

from enterprise.signals import parameter
from enterprise.signals import signal_base
from enterprise.signals import deterministic_signals
from enterprise.signals import utils
from enterprise.signals import gp_signals
from enterprise import constants as const
from enterprise.signals.signal_base import LogLikelihood

from enterprise_extensions import models as ee_models
from enterprise_extensions import model_utils as ee_model_utils
from enterprise_extensions import sampler as ee_sampler
from enterprise_extensions import blocks as ee_blocks
from enterprise_extensions import deterministic as ee_deterministic

from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc

In [4]:
nano11_pkl = '/home/nima/nanograv/11yr_factlike/NANOGrav_11yr_DE436.pickle'
with open(nano11_pkl, 'rb') as f:
    allpsrs=pickle.load(f)
    
noisefile = '/home/nima/nanograv/11yr_factlike/noisefiles/noisedict.json'
with open(noisefile, 'rb') as f:
    noisedict = json.load(f)
    
signs=[-1, 1]

In [ ]:
for psr in allpsrs:
    for sign in signs:
        
        outdir = "/home/nima/nanograv/11yr_factlike/bayesian_verification/individual_pulsars_fixed_sign/{}_{}/".format(psr.name, sign) 
        os.makedirs(outdir, exist_ok=True)
        
        t0min_sec = psr.toas.min() + 180*3600*24
        t0max_sec = psr.toas.max() - 180*3600*24

        t0min_mjd = t0min_sec/3600/24
        t0max_mjd = t0max_sec/3600/24


        t0 = parameter.Uniform(t0min_mjd, t0max_mjd)('ramp_t0')
        

        # This has 
        bwm_log10_As = parameter.LinearExp(-17, -10)


        ramp_wf = ee_deterministic.bwm_sglpsr_delay(log10_A=bwm_log10_As, t0=t0, sign=parameter.Constant(sign)('sign'))
        ramp = deterministic_signals.Deterministic(ramp_wf, name='ramp')

        s = ee_blocks.red_noise_block(psd='powerlaw', prior='uniform', components=30,
                                     logmin=-17, logmax=-10, Tspan=None)
        s += ee_blocks.white_noise_block(vary=False, inc_ecorr=True)
        s += ramp
        s += gp_signals.TimingModel()

        models = [s(psr)]
        pta = signal_base.PTA(models)


        print("Here are the parameters of the pta: {}".format(pta.params))

        pta.set_default_params(noisedict)
        # Save the pta parameters and prior distributions
        np.savetxt(outdir+'pars.txt',list(map(str, pta.param_names)), fmt='%s')
        np.savetxt(outdir+'priors.txt',list(map(lambda x: str(x.__repr__()), pta.params)), fmt='%s')

        x0 = np.hstack(p.sample() for p in pta.params)
        ndim = len(x0)

        # initial jump covariance matrix
        cov = np.diag(np.ones(ndim) * 0.1**2)
        sampler = ptmcmc(ndim, pta.get_lnlikelihood, pta.get_lnprior, cov,  outDir=outdir, resume=True)

        jp = ee_sampler.JumpProposal(pta)
        sampler.addProposalToCycle(jp.draw_from_prior, 30)

        sampler.sample(x0, int(1e5), SCAMweight=30, AMweight=50, DEweight=0)

Here are the parameters of the pta: [B1855+09_ramp_log10_A:LinearExp(pmin=-17, pmax=-10), B1855+09_red_noise_gamma:Uniform(pmin=0, pmax=7), B1855+09_red_noise_log10_A:LinearExp(pmin=-17, pmax=-10), ramp_t0:Uniform(pmin=53538.72366848987, pmax=57195.72889084464)]
Resuming run from chain file /home/nima/nanograv/11yr_factlike/bayesian_verification/individual_pulsars_fixed_sign/B1855+09_-1//chain_1.txt


/tmp/ipykernel_12544/3792533100.py:41: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  x0 = np.hstack(p.sample() for p in pta.params)


Adding DE jump with weight 0
Finished 99.00 percent in 1047.660962 s Acceptance rate = 0.242794
Run Complete
Here are the parameters of the pta: [B1855+09_ramp_log10_A:LinearExp(pmin=-17, pmax=-10), B1855+09_red_noise_gamma:Uniform(pmin=0, pmax=7), B1855+09_red_noise_log10_A:LinearExp(pmin=-17, pmax=-10), ramp_t0:Uniform(pmin=53538.72366848987, pmax=57195.72889084464)]
Finished 10.00 percent in 120.750564 s Acceptance rate = 0.2158Adding DE jump with weight 0
Finished 99.00 percent in 1206.928399 s Acceptance rate = 0.223303
Run Complete
Here are the parameters of the pta: [B1937+21_ramp_log10_A:LinearExp(pmin=-17, pmax=-10), B1937+21_red_noise_gamma:Uniform(pmin=0, pmax=7), B1937+21_red_noise_log10_A:LinearExp(pmin=-17, pmax=-10), ramp_t0:Uniform(pmin=53447.08924884693, pmax=57196.79511727041)]
Finished 10.00 percent in 208.401231 s Acceptance rate = 0.16159Adding DE jump with weight 0
Finished 99.00 percent in 2105.502023 s Acceptance rate = 0.216818
Run Complete
Here are the paramet